In [90]:
import pandas as pd
import json, os
from datetime import datetime
import numpy as np

In [126]:
providers = {
    'aws': 'aws-amazon-web-services-reports',
    'gcp': 'google-cloud-reports',
    'azure': 'windows-azure-reports'
}

Empty DataFrame
Columns: [timestamp, count, provider, cause]
Index: []

In [64]:
df = pd.DataFrame(columns={'timestamp': [], 'count' : [], 'provider': []})
for provider, reports in providers.items():
    path = os.getcwd() + "/down-detector/" + reports
    provider_html_pages = os.listdir(path)
    
    for provider_page in provider_html_pages:
        file = open(path + "/" + provider_page, "r")
        json_obj = json.load(file)
        counts_obj = json_obj["counts"]
        
        for count in counts_obj:
            timestamp = datetime.strptime(count['x'], '%Y-%m-%dT%H:%M:%S%z').timestamp()
            nr_report = count['y']
            df = df.append({'timestamp': timestamp, 'count': nr_report, 'provider': provider}, ignore_index=True)

In [81]:
df = df.sort_values('timestamp').reset_index(drop=True, inplace=False)
df.to_csv('down_detector_outage_count.csv')

In [163]:
df_reasons = pd.DataFrame(columns={'id': [], 'timestamp': [], 'count' : [], 'provider': [], 'cause': [], 'percent': []})

row_id = 0
for provider, reports in providers.items():
    path = os.getcwd() + "/down-detector/" + reports
    provider_html_pages = os.listdir(path)
    
    for provider_page in provider_html_pages:
        file = open(path + "/" + provider_page, "r")
        json_obj = json.load(file)
        counts_obj = json_obj["counts"]
        reasons_obj = json_obj.get("reasons")
        for count in counts_obj:
            timestamp = datetime.strptime(count['x'], '%Y-%m-%dT%H:%M:%S%z').timestamp()
            nr_report = count['y']
            if reasons_obj != None:
                for reason in reasons_obj:
                    cause = list(reason.keys())[0]
                    percent = float(list(reason.values())[0].split("%")[0]) / 100 
                    df_reasons = df_reasons.append({'id': row_id, 'timestamp': timestamp, 'count': nr_report, 'provider': provider, 'cause': cause, 'percent': percent}, ignore_index=True)
                    
                row_id += 1
            else:
                df_reasons = df_reasons.append({'id': row_id, 'timestamp': timestamp, 'count': nr_report, 'provider': provider, 'cause': np.NaN, 'percent': np.NaN}, ignore_index=True)
                row_id += 1

In [167]:
df_reasons.sort_values('timestamp').reset_index(drop=True, inplace=True)
df_reasons.to_csv('down_detector_reported_problem_count.csv', index=False)

In [168]:
df_reasons

id     timestamp count provider cause  percent
0          0  1.561334e+09     1      aws   NaN      NaN
1          1  1.561335e+09     2      aws   NaN      NaN
2          2  1.561335e+09     1      aws   NaN      NaN
3          3  1.561336e+09     1      aws   NaN      NaN
4          4  1.561337e+09     2      aws   NaN      NaN
...      ...           ...   ...      ...   ...      ...
20508  12528  1.525417e+09     0    azure   NaN      NaN
20509  12529  1.525418e+09     0    azure   NaN      NaN
20510  12530  1.525418e+09     0    azure   NaN      NaN
20511  12531  1.525419e+09     0    azure   NaN      NaN
20512  12532  1.525420e+09     0    azure   NaN      NaN

[20513 rows x 6 columns]